In [6]:
#Cleaned up Corona County Comparisons

In [7]:
import ipywidgets as widgets
from IPython import display

import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
import xlrd

from datetime import datetime

In [8]:

countydem_df = pd.read_excel(r'/Users/eileenyang/JupyterPython/Corona/uscountydemographics.xlsx')
covid_df = pd.read_csv(r'/Users/eileenyang/JupyterPython/Corona/covid_may25.csv')


In [9]:
countydem = countydem_df[['State','County','TotalPop','Poverty','Income','Unemployment']]
countydem2 = countydem.groupby(['State','County']).sum()
countydem2 = countydem2.reset_index()

c1=countydem2[['State','County','TotalPop']]
countydem3=countydem.groupby(['State','County']).mean()
countydem3=countydem3.reset_index()
c2=countydem3.drop(columns=['TotalPop'])

county = c1.merge(c2, on = ['State','County'])
county['County'] = county['County'].str.replace(r' County', '')


In [10]:
covid_df['date'] =  pd.to_datetime(covid_df['date'])
x = covid_df['date'].max()

covid_current = covid_df[covid_df['date']==x]
covid_current = covid_current.rename(columns={'state':'State','county':'County'})

covidcounty = covid_current.merge(county, on=['State','County'])


In [11]:
povstd = np.std(covidcounty['Poverty'])

incomestd = np.std(covidcounty['Income'])

unemploystd = np.std(covidcounty['Unemployment'])

In [12]:
idk = county.groupby(['State']).agg(list)
idk = idk[['County']]
idk = idk.reset_index()

idkt = idk.T

new_header = idkt.iloc[0] #grab the first row for the header
idkt = idkt[1:] #take the data less the header row
idkt.columns = new_header

idkt_dict = idkt.to_dict('list')

for key, value in idkt_dict.items():
    # do something with value
    idkt_dict[key] = value[0]

In [13]:
#interactive widget to run the function! aka the final product

from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual


def countycompare_pov(State, County):
        cc.options = idkt_dict[State]

        cond1 = covidcounty['County']==County
        cond2 = covidcounty['State']==State
        county_choose = covidcounty[(cond1) & (cond2)]       
        county_choose = county_choose.reset_index()

        pov = county_choose.at[0,'Poverty']
        pop = county_choose.at[0,'TotalPop']

        minpov = pov-povstd
        maxpov = pov+povstd

        minpop = 0.5*pop
        maxpop = 1.5*pop

        simcountiespov_cond1 = covidcounty['Poverty'].between(minpov,maxpov)
        simcountiespov_cond2 = covidcounty['TotalPop'].between(minpop,maxpop)
        simcounties_pov = covidcounty[(simcountiespov_cond1) & (simcountiespov_cond2)]

        simcounties_pov['dummy'] = 1
        county_choose['dummy'] = 1

        fig=plt.figure()

        bp = simcounties_pov.boxplot(column=['cases'], by=['dummy'], vert=False, grid=False)

        plt.xlabel("")
        plt.ylabel("")
        plt.title("COVID-19 Cases in " + County+" County, "+ State + " compared to Counties with Similar Poverty Rate")
        plt.suptitle("")

        ax = plt.gca()
        ax.axes.yaxis.set_ticks([])

        for i in [1]:
            plt.plot(county_choose['cases'], i, marker="x")
            plt.figure()

        
ss = widgets.Dropdown(options=idkt_dict.keys())
init = ss.value
cc = widgets.Dropdown(options=idkt_dict[init])

w4=interactive(countycompare_pov, State=ss, County=cc)
display(w4)

interactive(children=(Dropdown(description='State', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Cali…